In [1]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from scipy.io import loadmat
import ipdb
import os
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

# from tensorflow.compat.v1 import ConfigProto
# from tensorflow.compat.v1 import InteractiveSession

# config = ConfigProto()
# config.gpu_options.allow_growth = True
# session = InteractiveSession(config=config)
class CnnLstm():
    def __init__(self, classes, filters=32, kernel_size=40, pool_size=4, drop_rate=0.1, hidden_lstm=128):
        """
        :param classes: 做多少分类任务
        :param filters: 卷积长度
        :param kernel_size: 卷积核的大小
        :param pool_size: 池化层核的大小
        :param drop_rate: drop out 的比例
        :param hidden_lstm: lstm中的cell的个数
        """

        self.classes = classes
        self.filters = filters
        self.kernel_size = kernel_size
        self.pool_size = pool_size
        self.drop_rate = drop_rate
        self.hidden_lstm = hidden_lstm
        self.model = self.build_model()
#         print(self.model.summary())

    def build_model(self):
        model = keras.Sequential()
        model.add(keras.layers.Conv1D(filters=self.filters, kernel_size=self.kernel_size, activation=keras.activations.relu))
        model.add(keras.layers.BatchNormalization())
        model.add(keras.layers.MaxPooling1D(pool_size=self.pool_size))
        model.add(keras.layers.Dropout(rate=self.drop_rate))

        model.add(keras.layers.Conv1D(filters=self.filters, kernel_size=self.kernel_size, activation=keras.activations.relu))
        model.add(keras.layers.BatchNormalization())
        model.add(keras.layers.MaxPooling1D(pool_size=self.pool_size))
        model.add(keras.layers.Dropout(rate=self.drop_rate))

        model.add(keras.layers.LSTM(self.hidden_lstm, activation=keras.activations.tanh, return_sequences=True))
        model.add(keras.layers.LSTM(self.hidden_lstm, activation=keras.activations.tanh))

        model.add(keras.layers.Dense(self.classes, activation=keras.activations.relu))
        return model


In [2]:
def LoadData(filepath):
#     print(filepath)
    time=loadmat(filepath)
    time=time['ppg']
    listslice=filepath.split('-')
    label=listslice[2][:-4]
    return time,label

In [3]:
import glob,os
def trainset(setdir):
    datas=[]
    labels=[]
    filepaths=glob.glob(os.path.join(setdir, '*.mat')) 
    length=len(filepaths)
    print('Found {} data names'.format(length))
    for i in range(length):
        time,label=LoadData(filepaths[i])
        datas.append(time)
        labels.append(label)
    datas=np.asarray(datas)
    labels=np.asarray(labels,dtype=float)
    datas=datas.reshape((datas.shape[0],datas.shape[2],datas.shape[1]))
    labels=labels.reshape((labels.shape[0],1))
    print('datas shape: ',datas.shape)
    print('labels shape: ',labels.shape)
    return datas,labels
datas,labels=trainset('/home/wcj/ReferenceProject/PPGnet/ppghr')        

Found 1622 data names
datas shape:  (1622, 1000, 1)
labels shape:  (1622, 1)


In [ ]:
EPOCHS=100
BATCH_SIZE=300
# datas=tf.data.Dataset.from_tensor_slices(datas)

# datas=np.random.randn(100,1000,1)
# # train_y=np.random.random_integers(2,size=(100,1))
# labels=np.random.randint(1,size=(100,1))
# print('datas shape: ',datas.shape)
# print('labels shape: ',labels.shape)
# print(labels[0])
# print(datas[0])
cnnlstm = CnnLstm(classes=1,hidden_lstm=1000)
cnnlstm.model.compile(optimizer='RMSprop',    # 这里是优化器的选择
                          loss='mean_absolute_error'  # 损失函数的选择
                                )                              # 训练
# cnnlstm.model.summary()
cnnlstm.model.fit(datas, labels, epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=1)

Train on 1622 samples
Epoch 1/100
1622/1622 [==============================] - 4s 2ms/sample - loss: 120.7793
Epoch 2/100
1622/1622 [==============================] - 1s 613us/sample - loss: 104.2165
Epoch 3/100
1622/1622 [==============================] - 1s 631us/sample - loss: 97.0596
Epoch 4/100
1622/1622 [==============================] - 1s 625us/sample - loss: 88.5966
Epoch 5/100
1622/1622 [==============================] - 1s 625us/sample - loss: 81.5057
Epoch 6/100
1622/1622 [==============================] - 1s 618us/sample - loss: 75.7539
Epoch 7/100
1622/1622 [==============================] - 1s 623us/sample - loss: 70.3273
Epoch 8/100
1622/1622 [==============================] - 1s 642us/sample - loss: 64.9179
Epoch 9/100
1622/1622 [==============================] - 1s 642us/sample - loss: 59.8604
Epoch 10/100
1622/1622 [==============================] - 1s 630us/sample - loss: 54.8807
Epoch 11/100
1622/1622 [==============================] - 1s 619us/sample - loss: 50.48

In [20]:
cnnlstm.model.fit(datas, labels, epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=1)

Train on 1622 samples
Epoch 1/100
1622/1622 [==============================] - 1s 652us/sample - loss: 8.1870
Epoch 2/100
1622/1622 [==============================] - 1s 631us/sample - loss: 8.8758
Epoch 3/100
1622/1622 [==============================] - 1s 634us/sample - loss: 8.4764
Epoch 4/100
1622/1622 [==============================] - 1s 645us/sample - loss: 7.6474
Epoch 5/100
1622/1622 [==============================] - 1s 644us/sample - loss: 9.0493
Epoch 6/100
1622/1622 [==============================] - 1s 642us/sample - loss: 8.2454
Epoch 7/100
1622/1622 [==============================] - 1s 645us/sample - loss: 7.8403
Epoch 8/100
1622/1622 [==============================] - 1s 655us/sample - loss: 7.0589
Epoch 9/100
1622/1622 [==============================] - 1s 664us/sample - loss: 7.1819
Epoch 10/100
1622/1622 [==============================] - 1s 662us/sample - loss: 7.8999
Epoch 11/100
1622/1622 [==============================] - 1s 660us/sample - loss: 7.8895
Epoch 12

Epoch 93/100
1622/1622 [==============================] - 1s 720us/sample - loss: 1.9952
Epoch 94/100
1622/1622 [==============================] - 1s 735us/sample - loss: 2.0967
Epoch 95/100
1622/1622 [==============================] - 1s 714us/sample - loss: 2.2075
Epoch 96/100
1622/1622 [==============================] - 1s 734us/sample - loss: 2.2916
Epoch 97/100
1622/1622 [==============================] - 1s 707us/sample - loss: 2.3077
Epoch 98/100
1622/1622 [==============================] - 1s 718us/sample - loss: 2.0494
Epoch 99/100
1622/1622 [==============================] - 1s 723us/sample - loss: 1.8211
Epoch 100/100
1622/1622 [==============================] - 1s 737us/sample - loss: 1.9964


In [21]:
cnnlstm.model.fit(datas, labels, epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=1)

Train on 1622 samples
Epoch 1/100
1622/1622 [==============================] - 1s 742us/sample - loss: 2.2186
Epoch 2/100
1622/1622 [==============================] - 1s 705us/sample - loss: 2.0508
Epoch 3/100
1622/1622 [==============================] - 1s 707us/sample - loss: 1.8120
Epoch 4/100
1622/1622 [==============================] - 1s 663us/sample - loss: 2.6043
Epoch 5/100
1622/1622 [==============================] - 1s 688us/sample - loss: 1.8648
Epoch 6/100
1622/1622 [==============================] - 1s 664us/sample - loss: 2.0059
Epoch 7/100
1622/1622 [==============================] - 1s 620us/sample - loss: 1.9857
Epoch 8/100
1622/1622 [==============================] - 1s 625us/sample - loss: 1.5347
Epoch 9/100
1622/1622 [==============================] - 1s 626us/sample - loss: 1.8658
Epoch 10/100
1622/1622 [==============================] - 1s 635us/sample - loss: 2.4675
Epoch 11/100
1622/1622 [==============================] - 1s 662us/sample - loss: 2.0924
Epoch 12

Epoch 93/100
1622/1622 [==============================] - 1s 745us/sample - loss: 1.3570
Epoch 94/100
1622/1622 [==============================] - 1s 748us/sample - loss: 1.3289
Epoch 95/100
1622/1622 [==============================] - 1s 731us/sample - loss: 1.3660
Epoch 96/100
1622/1622 [==============================] - 1s 736us/sample - loss: 1.3355
Epoch 97/100
1622/1622 [==============================] - 1s 735us/sample - loss: 1.2509
Epoch 98/100
1622/1622 [==============================] - 1s 787us/sample - loss: 1.0295
Epoch 99/100
1622/1622 [==============================] - 1s 771us/sample - loss: 1.3330
Epoch 100/100
1622/1622 [==============================] - 1s 756us/sample - loss: 1.2873


In [22]:
opt=keras.optimizers.RMSprop(learning_rate=0.0001)
cnnlstm.model.compile(optimizer=opt,    # 这里是优化器的选择
                          loss='mean_absolute_error'  # 损失函数的选择
                                )                              # 训练
# cnnlstm.model.summary()
cnnlstm.model.fit(datas, labels, epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=1)

Train on 1622 samples
Epoch 1/100
1622/1622 [==============================] - 4s 3ms/sample - loss: 0.8346
Epoch 2/100
1622/1622 [==============================] - 1s 650us/sample - loss: 0.6896
Epoch 3/100
1622/1622 [==============================] - 1s 660us/sample - loss: 0.6334
Epoch 4/100
1622/1622 [==============================] - 1s 657us/sample - loss: 0.5898
Epoch 5/100
1622/1622 [==============================] - 1s 690us/sample - loss: 0.6083
Epoch 6/100
1622/1622 [==============================] - 1s 698us/sample - loss: 0.5874
Epoch 7/100
1622/1622 [==============================] - 1s 698us/sample - loss: 0.5570
Epoch 8/100
1622/1622 [==============================] - 1s 657us/sample - loss: 0.5685
Epoch 9/100
1622/1622 [==============================] - 1s 703us/sample - loss: 0.5404
Epoch 10/100
1622/1622 [==============================] - 1s 675us/sample - loss: 0.5256
Epoch 11/100
1622/1622 [==============================] - 1s 672us/sample - loss: 0.5365
Epoch 12/1

Epoch 93/100
1622/1622 [==============================] - 1s 762us/sample - loss: 0.3488
Epoch 94/100
1622/1622 [==============================] - 1s 766us/sample - loss: 0.3317
Epoch 95/100
1622/1622 [==============================] - 1s 739us/sample - loss: 0.3247
Epoch 96/100
1622/1622 [==============================] - 1s 738us/sample - loss: 0.3109
Epoch 97/100
1622/1622 [==============================] - 1s 729us/sample - loss: 0.3032
Epoch 98/100
1622/1622 [==============================] - 1s 736us/sample - loss: 0.3144
Epoch 99/100
1622/1622 [==============================] - 1s 766us/sample - loss: 0.3148
Epoch 100/100
1622/1622 [==============================] - 1s 729us/sample - loss: 0.3083


In [23]:
opt=keras.optimizers.RMSprop(learning_rate=0.00001)
cnnlstm.model.compile(optimizer=opt,    # 这里是优化器的选择
                          loss='mean_absolute_error'  # 损失函数的选择
                                )                              # 训练
# cnnlstm.model.summary()
cnnlstm.model.fit(datas, labels, epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=1)

Train on 1622 samples
Epoch 1/100
1622/1622 [==============================] - 5s 3ms/sample - loss: 0.3078
Epoch 2/100
1622/1622 [==============================] - 1s 674us/sample - loss: 0.3016
Epoch 3/100
1622/1622 [==============================] - 1s 673us/sample - loss: 0.3014
Epoch 4/100
1622/1622 [==============================] - 1s 677us/sample - loss: 0.3039
Epoch 5/100
1622/1622 [==============================] - 1s 707us/sample - loss: 0.2865
Epoch 6/100
1622/1622 [==============================] - 1s 735us/sample - loss: 0.2756
Epoch 7/100
1622/1622 [==============================] - 1s 696us/sample - loss: 0.3014s - loss: 0.301
Epoch 8/100
1622/1622 [==============================] - 1s 695us/sample - loss: 0.2930
Epoch 9/100
1622/1622 [==============================] - 1s 745us/sample - loss: 0.2825
Epoch 10/100
1622/1622 [==============================] - 1s 693us/sample - loss: 0.2895
Epoch 11/100
1622/1622 [==============================] - 1s 678us/sample - loss: 0.

In [24]:
####################
# evalation
####################
test_datas,test_labels=trainset('/home/wcj/ReferenceProject/PPGnet/12thData')     
prediction=cnnlstm.model.evaluate(test_datas,test_labels,batch_size=10)

Found 146 data names
datas shape:  (146, 1000, 1)
labels shape:  (146, 1)
146/1 [=======================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

In [25]:
test_datas_10,test_labels_10=trainset('/home/wcj/ReferenceProject/PPGnet/testdata')     
prediction=cnnlstm.model.evaluate(test_datas_10,test_labels_10,batch_size=100)

Found 1328 data names
datas shape:  (1328, 1000, 1)
labels shape:  (1328, 1)
1328/1 [===================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

In [29]:
######################
# save model
######################
cnnlstm.model.save('epoch_300-batch_size-100.h5')
# cnnlstm.model.summary()

In [30]:
###################
# load model
###################

#重新创建完全相同的模型，包括其权重和优化程序
new_model = keras.models.load_model('epoch_300-batch_size-100.h5')

# 显示网络结构
new_model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_10 (Conv1D)           multiple                  1312      
_________________________________________________________________
batch_normalization_10 (Batc multiple                  128       
_________________________________________________________________
max_pooling1d_10 (MaxPooling multiple                  0         
_________________________________________________________________
dropout_10 (Dropout)         multiple                  0         
_________________________________________________________________
conv1d_11 (Conv1D)           multiple                  40992     
_________________________________________________________________
batch_normalization_11 (Batc multiple                  128       
_________________________________________________________________
max_pooling1d_11 (MaxPooling multiple                 

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from scipy.io import loadmat
import ipdb

In [37]:
a=np.array([1,2,3,4,5])
a=np.expand_dims(a,axis=0)
print(a.shape)
b=np.array([[6,7,8,9,10]])
print(b.shape)
c=np.concatenate((a,b),axis=0)
d=np.array([[11,12,13,14,15]])
e=np.concatenate((c,d),axis=0)
e.shape[0]

(1, 5)
(1, 5)


3

In [23]:
c

array([[ 1,  2,  3,  4,  5],
       [ 6,  7,  8,  9, 10]])

In [31]:
filepath="/home/wcj/ReferenceProject/PPGnet/ppghr/1-104-150.9534.mat"
time=loadmat(filepath)
time=time['ppg']
ds1 = tf.data.Dataset.from_tensor_slices(time)
ds1

<TensorSliceDataset shapes: (1000,), types: tf.float64>

In [32]:
ds6 = tf.data.Dataset.list_files("/home/wcj/ReferenceProject/PPGnet/ppghr/*.mat")
for file in ds6.take(5):
    print(file)

tf.Tensor(b'/home/wcj/ReferenceProject/PPGnet/ppghr/11-78-168.4492.mat', shape=(), dtype=string)
tf.Tensor(b'/home/wcj/ReferenceProject/PPGnet/ppghr/9-140-150.1581.mat', shape=(), dtype=string)
tf.Tensor(b'/home/wcj/ReferenceProject/PPGnet/ppghr/1-93-151.919.mat', shape=(), dtype=string)
tf.Tensor(b'/home/wcj/ReferenceProject/PPGnet/ppghr/6-111-138.587.mat', shape=(), dtype=string)
tf.Tensor(b'/home/wcj/ReferenceProject/PPGnet/ppghr/4-129-164.0625.mat', shape=(), dtype=string)


In [38]:
# for i,(data,label) in enumerate(ds6.map(tf.py_function(LoadData)).ake(2)):
#     print(data.shape)
#     print(label)
    
tf.py_function(LoadData,["/home/wcj/ReferenceProject/PPGnet/ppghr/1-104-150.9534.mat"],[tf.uint16])

InvalidArgumentError: TypeError: 'DType' object is not callable
Traceback (most recent call last):

  File "/home/wcj/anaconda2/envs/pytorch/lib/python3.7/site-packages/tensorflow_core/python/ops/gen_script_ops.py", line 52, in eager_py_func
    token, "Tout", Tout)

tensorflow.python.eager.core._FallbackException: This function does not handle the case of the path where all inputs are not already EagerTensors.


During handling of the above exception, another exception occurred:


Traceback (most recent call last):

  File "/home/wcj/anaconda2/envs/pytorch/lib/python3.7/site-packages/tensorflow_core/python/ops/script_ops.py", line 219, in __call__
    return func(device, token, args)

  File "/home/wcj/anaconda2/envs/pytorch/lib/python3.7/site-packages/tensorflow_core/python/ops/script_ops.py", line 113, in __call__
    ret = self._func(*args)

  File "<ipython-input-30-a3191ecfbcee>", line 3, in LoadData
    filepath=tf.string(filepath).values

TypeError: 'DType' object is not callab

In [5]:
###########################
# https://stackoverflow.com/questions/46890387/how-to-read-mat-file-format-in-tensorflow?rq=1 
##########################
def read_mat(filepath):   
    def _read_mat(filepath):
        matfile = loadmat(filepath)
        data0 = matfile['ppg']
#         data1 = matfile['data1']
#         data2 = mat_contents['data2']
#         shape0 = matfile['data0'].shape
        listslice=filepath.split('-')
        label=listslice[2][:-4]
        return data0,label
#         return data0, data1, data2, np.asarray(shape0)

    output = tf.py_function(_read_mat, [filepath], [tf.float32])
    ipdb.set_trace()
    shape = output[0]
    data0 = tf.reshape(output[0], shape)
    data1 = tf.reshape(output[1], shape)
    data2 = tf.reshape(output[2], shape)
    return data0, data1, data2

dataset = tf.data.Dataset.list_files('/home/wcj/ReferenceProject/PPGnet/ppghr/*.mat')
dataset = dataset.map(read_mat, num_parallel_calls=16)
dataset = dataset.repeat(100)
dataset = dataset.batch(8)
dataset = dataset.prefetch(8)
iterator = dataset.make_initializable_iterator()
sess = tf.Session()
sess.run(iterator.initializer)
values = sess.run(iterator.get_next())

> /tmp/tmp9x2comq1.py(26)tf__read_mat()
     25         ipdb.set_trace()
---> 26         shape = output[0]
     27         data0 = ag__.converted_call(tf.reshape, read_mat_scope.callopts, (output[0], shape), None, read_mat_scope)

ipdb> output[0]
<tf.Tensor 'EagerPyFunc:0' shape=<unknown> dtype=float32>
ipdb> output[0].shape
TensorShape(None)
ipdb> n
> /tmp/tmp9x2comq1.py(27)tf__read_mat()
     26         shape = output[0]
---> 27         data0 = ag__.converted_call(tf.reshape, read_mat_scope.callopts, (output[0], shape), None, read_mat_scope)
     28         data1 = ag__.converted_call(tf.reshape, read_mat_scope.callopts, (output[1], shape), None, read_mat_scope)

ipdb> n
TypeError: Value passed to parameter 'shape' has DataType float32 not in list of allowed values: int32, int64
> /tmp/tmp9x2comq1.py(27)tf__read_mat()
     26         shape = output[0]
---> 27         data0 = ag__.converted_call(tf.reshape, read_mat_scope.callopts, (output[0], shape), None, read_mat_scope)
     28    

TypeError: in converted code:

    <ipython-input-4-c7f884d25974>:19 read_mat  *
        data0 = tf.reshape(output[0], shape)
    /home/wcj/anaconda2/envs/pytorch/lib/python3.7/site-packages/tensorflow_core/python/ops/array_ops.py:131 reshape
        result = gen_array_ops.reshape(tensor, shape, name)
    /home/wcj/anaconda2/envs/pytorch/lib/python3.7/site-packages/tensorflow_core/python/ops/gen_array_ops.py:8117 reshape
        "Reshape", tensor=tensor, shape=shape, name=name)
    /home/wcj/anaconda2/envs/pytorch/lib/python3.7/site-packages/tensorflow_core/python/framework/op_def_library.py:631 _apply_op_helper
        param_name=input_name)
    /home/wcj/anaconda2/envs/pytorch/lib/python3.7/site-packages/tensorflow_core/python/framework/op_def_library.py:60 _SatisfiesTypeConstraint
        ", ".join(dtypes.as_dtype(x).name for x in allowed_list)))

    TypeError: Value passed to parameter 'shape' has DataType float32 not in list of allowed values: int32, int64


In [37]:
BATCH_SIZE=100
ds_train = tf.data.Dataset.list_files("/home/wcj/ReferenceProject/PPGnet/ppghr/*.mat") \
           .map(LoadData, num_parallel_calls=tf.data.experimental.AUTOTUNE) \
           .shuffle(buffer_size = 1000).batch(BATCH_SIZE) \
           .prefetch(tf.data.experimental.AUTOTUNE)  

    

TypeError: in converted code:

    <ipython-input-30-a3191ecfbcee>:3 LoadData  *
        filepath=tf.string(filepath).values

    TypeError: 'DType' object is not callable
